# Задание
Цель - исследовать возможности pandas по работе с данными.
Скачайте архив с https://www.dropbox.com/s/9ymqw9sofaameql/trainingData_JSON_chunk1.json.gz?dl=0
Архив представляет из себя построчный список JSON объектов.
Каждый JSON представляет из себя словарь-описание игровой ситуации (есть ключи по герою, по оппоненту, по существам на столе и т.д.)

Ниже пример кода как их прочиать в pd.DataFrame (поля можно выбрать какие хотите)
```
df_cards = []
for filename in glob.glob('../*_JSON_*'):
    with open(filename) as fin:
        for record in tqdm(map(json.loads, fin)):
            for card in record['player'].get('hand', []):
                card['turn'] = record['turn']
                card['gamestate_id'] = record['gamestate_id']
                card['hero'] = record['player']['hero']['hero_card_id']
                df_cards.append(card)
df = pd.DataFrame.from_records(df_cards)
```
У героев тут стоят id, вот их отображения в классы:

```
heroes = {
    25: 'shaman',
    326: 'hunter',
    390: 'warlock',
    494: 'mage',
    612: 'druid',
    754: 'rogue',
    798: 'priest',
    981: 'paladin',
    1235: 'warrior'
}
```

## Задание 1
Сделать таблицу матчапов 9х9 где по обоим осям названия героев а в ячейке i, j - процент побед(средний decision) героя i против героя j

## Задание 2
По каждой уникальной карте сделать таблицу с полями:
name	poisonous	shield	stealth	taunt	turn	type	windfury	hero_name
Пример: https://snag.gy/2rFpbz.jpg

## Задание 3
Посчитать средний винрейт(средний decision) по каждой карте (среди колод, где данная карта присутствует)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pylab inline
import numpy as np
import pandas as pd
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


#### reading data

In [4]:
from tqdm import tqdm
import json
df_cards = []
filename = '../data/trainingData_JSON_chunk1.json'
with open(filename) as fin:
    for record in tqdm(map(json.loads, fin)):
        for card in record['player'].get('hand', []):
            card['turn'] = record['turn']
            card['gamestate_id'] = record['gamestate_id']
            card['hero'] = record['player']['hero']['hero_card_id']
            card['opponent_hero'] = record['opponent']['hero']['hero_card_id']
            #card['opponent_name'] = record['opponent']['played_cards']['name']
            card['decision'] = record['decision']
            df_cards.append(card)
df = pd.DataFrame.from_records(df_cards)

500000it [00:45, 11085.99it/s]


In [5]:
df.head()

,attack,charge,crystals_cost,decision,durability,forgetful,freeze,freezing,gamestate_id,hero,...,id,name,opponent_hero,poisonous,shield,stealth,taunt,turn,type,windfury
0,5.0,False,7,1,NaN,False,NaN,False,4687346,981,...,2137,Guardian of Kings,798,False,False,False,False,14,MINION,False
1,2.0,False,2,1,NaN,False,NaN,False,4687346,981,...,1902,Murloc Tidehunter,798,False,False,False,False,14,MINION,False
2,4.0,NaN,4,1,2.0,False,False,NaN,4687346,981,...,172,Truesilver Champion,798,NaN,NaN,NaN,NaN,14,WEAPON,NaN
3,3.0,False,2,1,NaN,False,NaN,False,4687346,981,...,1155,Acidic Swamp Ooze,798,False,False,False,False,14,MINION,False
4,6.0,False,7,1,NaN,False,NaN,False,4687346,981,...,300,Stormwind Champion,798,False,False,False,False,14,MINION,False


In [16]:
df.shape

(2304421, 20)

### preparing dataframe for work

In [6]:
df_for_work = df[['decision', 'name', 'poisonous', 'shield', 'stealth',
                  'taunt', 'turn', 'type', 'windfury']]
hero_card_id = { 25: 'shaman', 326: 'hunter', 390: 'warlock', 494: 'mage', 
                612: 'druid', 754: 'rogue', 798: 'priest', 981: 'paladin', 
                1235: 'warrior'}
df_for_work['hero'] = df['hero'].map(hero_card_id)
df_for_work['opponent_hero'] = df['opponent_hero'].map(hero_card_id)
df_for_work.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,decision,name,poisonous,shield,stealth,taunt,turn,type,windfury,hero,opponent_hero
0,1,Guardian of Kings,False,False,False,False,14,MINION,False,paladin,priest
1,1,Murloc Tidehunter,False,False,False,False,14,MINION,False,paladin,priest
2,1,Truesilver Champion,NaN,NaN,NaN,NaN,14,WEAPON,NaN,paladin,priest
3,1,Acidic Swamp Ooze,False,False,False,False,14,MINION,False,paladin,priest
4,1,Stormwind Champion,False,False,False,False,14,MINION,False,paladin,priest


# Задание 1

In [8]:
df_for_work.pivot_table(index='hero', columns='opponent_hero', values='decision')

opponent_hero,druid,hunter,mage,paladin,priest,rogue,shaman,warlock,warrior
hero,,,,,,,,,
druid,0.481535,0.493059,0.499659,0.495676,0.493580,0.498933,0.481967,0.509310,0.500472
hunter,0.511360,0.484766,0.501777,0.502375,0.498902,0.497866,0.486868,0.493590,0.524871
mage,0.472710,0.490263,0.478505,0.500101,0.492019,0.495812,0.483165,0.480531,0.462521
paladin,0.481438,0.489381,0.509378,0.497768,0.513639,0.488758,0.485734,0.489486,0.476657
priest,0.510035,0.478049,0.506536,0.505544,0.511002,0.503174,0.498279,0.497251,0.485901
rogue,0.480907,0.481082,0.499254,0.484115,0.481939,0.484539,0.482277,0.496418,0.476980
shaman,0.501467,0.493579,0.514331,0.502595,0.496285,0.504875,0.497927,0.510762,0.504431
warlock,0.492162,0.506332,0.510062,0.491555,0.494365,0.506532,0.496003,0.505349,0.482116
warrior,0.490658,0.493252,0.520759,0.497189,0.521228,0.501505,0.501851,0.480517,0.521044


# Задание 2

In [71]:
df_show = pd.DataFrame()
for elem in df_for_work['name'].unique():
    mask = np.in1d(df_for_work['name'].values, [elem])
    df_show = df_show.append(df_for_work.loc[mask].iloc[0][['name', 'poisonous', 'shield',
                                                            'stealth','taunt', 'turn', 'type', 
                                                            'windfury', 'hero']], ignore_index=True)

In [72]:
df_show

,hero,name,poisonous,shield,stealth,taunt,turn,type,windfury
0,paladin,Guardian of Kings,0.0,0.0,0.0,0.0,14.0,MINION,0.0
1,paladin,Murloc Tidehunter,0.0,0.0,0.0,0.0,14.0,MINION,0.0
2,paladin,Truesilver Champion,NaN,NaN,NaN,NaN,14.0,WEAPON,NaN
3,paladin,Acidic Swamp Ooze,0.0,0.0,0.0,0.0,14.0,MINION,0.0
4,paladin,Stormwind Champion,0.0,0.0,0.0,0.0,14.0,MINION,0.0
5,rogue,Chillwind Yeti,0.0,0.0,0.0,0.0,7.0,MINION,0.0
6,rogue,Assassin's Blade,NaN,NaN,NaN,NaN,7.0,WEAPON,NaN
7,rogue,Assassinate,NaN,NaN,NaN,NaN,7.0,SPELL,NaN
8,rogue,Deadly Poison,NaN,NaN,NaN,NaN,7.0,SPELL,NaN
9,druid,Ironbark Protector,0.0,0.0,0.0,1.0,5.0,MINION,0.0


# Задание 3 
### не совсем понял что означает "среди колод, где данная карта присутствует"

In [19]:
pd.crosstab(df_for_work['decision'], df_for_work['hero'], normalize='columns')

hero,druid,hunter,mage,paladin,priest,rogue,shaman,warlock,warrior
decision,,,,,,,,,
0,0.505287,0.499816,0.516346,0.507355,0.50009,0.515083,0.49696,0.501951,0.494784
1,0.494713,0.500184,0.483654,0.492645,0.49991,0.484917,0.50304,0.498049,0.505216
